This is the prototype of the Mean Field

There will be three main modules:

- 1) Agent

- 2) MeanField

- 3) BlackBoard

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import scipy as sp
import ode
from sliding_window import *

### Agent

In [3]:
class Agent:
    
    def __init__(self, blackboard, state_indices, control_indices):
        '''
        state_indices (list of integers): This list tells which states pertain to this agent. e.g. [1,2] would 
        tell us that states 1 and 2 pertain to this agent.
        
        A word on notation:  The notation used for the methods of the agent is:  
            - If it is a partial derivative: <denominator>_rhs_H_<type of hamiltonian (l, mf, or s)>_<nou or u>.  e.g., 
            "qp_rhs_H_l_u" denotes the partial derivative with respect to q and p of the terms in the local Hamiltonian that contain control variables.
            - If it is a hamiltonian: H_<type of hamiltonian (l, mf, or s)>_<nou or u>.  e.g. "H_mf_nou" denotes the mean field hamiltonian
            with terms not containing u.
        '''
        self.state_indices = state_indices
        self.control_indices = control_indices
        self.bb = blackboard

        # Inputs for numerical propagator
        # qp_vec is going to be [q_s, p_l, p_mf], so it will have dimension = 3*state_dim

        self.q_s_0 = np.array([0])
        self.p_l_0 = np.array([0])
        self.p_mf_0 = np.array([0])
        self.u_s_0 = np.array([0])
        self.qpu_vec = np.hstack([self.q_s_0, self.p_l_0, self.p_mf_0, self.u_s_0])
        self.q_s_dot = np.array([0])  # must have same dimensions as q_s
        self.state_dim = 1
        self.Gamma = 1
        self.gamma = 1  # function is inputted by the user to compute this.
        self.sync = None # gets filled in when Synchronizer class is initialized
        self.name='Agent1'

        # Inputs for numerical integration
        self.integrateTol = 10**-5
        self.integrateMaxIter = 400

        # Inputs for sliding window
        self.t_0 = 0 
        self.T =  2
        self.K=10

        self.t_terminal = 4
        self.n_s = 10

        self.validate_dimensions()
        
    def compute_gamma(self):
        # if only one agent, then gamma = 1
        if len(self.bb.agents) == 1:
            return 1
        q_s, p_l, p_mf, u_s = self.qpu_vec
        q_s_dot = self.q_s_dot
        num = self.L_l(q_s, q_s_dot, u_s)
        denom = 0

        assert len(self.bb.agents) != 0, 'Add agents to your blackboard by calling bb.update_q_p_u_dict(<agent>)'
        for agent in self.bb.agents:
            denom += agent.L_l(q_s, q_s_dot, u_s)
        self.gamma = float(num)/float(denom)
        

    def validate_dimensions(self):
        # TODO: move to parent class "SlidingWindow"
        assert len(self.state_indices) == self.state_dim, 'state dimensions are not consistent.  dimension of state indices is '+str(len(self.state_indices)) +' and state_dim is '+str(self.state_dim)
        assert len(self.control_indices) == len(self.u_s_0), 'control dimensions are not consistent.  dimension of control_indices is '+str(len(self.control_indices)) +' and len(u_0) is '+str(len(self.u_s_0))
        assert len(self.qpu_vec) == 3*self.state_dim + len(self.control_indices), ' control and state dimensions are not consistent with qpu_vec : length of qpu_vec is '+str(len(self.qpu_vec))+ ' and 3*self.state_dim + len(self.control_indices) is ' + str(3*self.state_dim + len(self.control_indices))
    
    '''
    TODO:  
    Add an assertion to check that the dimension of q_s_0, p_mf_0, and u_0:
        - the dimension of state_dim
        - state_indices and control_indices set upon initiation of the Agent
    '''
    
    def L_l(self, q_s, q_s_dot, u_s):
        return 1
    
    def L_l_q_dot(self, q_s, q_s_dot, u_s):
        return q_s
    
    def H_l_nou(self, q_s, p_l, lambda_l):
        return 1

    def p_rhs_H_l_nou(self, q_s, p_l, lambda_l):
        return np.array(p_l)
    
    def q_rhs_H_l_nou(self, q_s, p_l, lambda_l):
        return np.array(p_l)
    
    # There should be one of these defined for each control variable

    def H_l_u_1(self, q_s, p_s):
        return 1
    
    def H_l(self, q_s, p_l, lambda_l, u_s):
        # used in "Construct local Hamiltonian of agent i"
        H_l = self.H_l_nou(q_s, p_l, lambda_l)
        H_l = H_l + self.H_l_u_1(q_s, p_s)*u_s[0]
        return H_l
            
    def compute_lambdas(self, q_s, p_l, u_l):
        # not implemented yet
        return np.ones((1,self.state_dim))
    
    def p_rhs_H_l_u(self, q_s, p_l):
        return np.array(p_l)
    
    def q_rhs_H_l_u(self, q_s, p_l):
        return np.array(p_l)
    
    def H_l_D(self, q_lD, p_lD):
        return np.array(q_lD).dot(p_lD)
        
    def L_l_D(self, q_lD, p_lD):
        # return scalar
        return 1
        
    def L_l_D_q_Dot(self, q_l_D, q_l_D_Dot):
        # return  1-D array of dimension 1 by state_dim,
        # each q_lD is a 1-D array of size 1 by state_dim array
        return 1

    def qp_rhs(self, t, qp_vec, **kwargs):
        # u_s is constant (because of causality, remember?)
        u_s = kwargs['u_0']
        state_dim = kwargs['state_dim']
        q_mf = kwargs['q_mf']
        u_mf = kwargs['u_mf']
        
        # TODO:  get a kwargs working for lambda_l
        lambda_l = 0 # kwargs['lambda_l']
        q_s = qp_vec[:state_dim]
        p_l = qp_vec[state_dim:2*state_dim]
        p_mf = qp_vec[2*state_dim:]
        
        qp_rhs_H_mf = self.qp_rhs_H_mf(q_mf, p_mf, u_mf)
        q_rhs_H_mf = qp_rhs_H_mf[:state_dim]
        p_rhs_H_mf = qp_rhs_H_mf[state_dim:]

        qp_rhs_H_l = self.qp_rhs_H_l(q_s, p_l, u_s, lambda_l)
        q_rhs_H_l = qp_rhs_H_l[:state_dim]
        p_rhs_H_l = qp_rhs_H_l[state_dim:]

        q_s_dot = self.gamma*q_rhs_H_mf + (1-self.gamma)*q_rhs_H_l
        p_mf_dot = p_rhs_H_mf
        p_l_dot = -1*p_rhs_H_l
        
        return np.concatenate([q_s_dot, p_l_dot, p_mf_dot])
    
    def qp_rhs_H_l(self, q_s, p_l, u_s, lambda_l):
        #TODO: there is one lambda_l per constraint. need to work out dimensions.
        q_rhs_H_l = self.q_rhs_H_l_nou(q_s, p_l, lambda_l) + sum([self.q_rhs_H_l_u(q_s, p_l)*u_s_i for u_s_i in u_s])
        p_rhs_H_l = self.p_rhs_H_l_nou(q_s, p_l, lambda_l) + sum([self.p_rhs_H_l_u(q_s, p_l)*u_s_i for u_s_i in u_s])
        return np.concatenate([q_rhs_H_l, p_rhs_H_l])

    def u_rhs(self, t, u_vec, **kwargs):
        return -1*self.Gamma*np.zeros(np.shape(u_vec))
    
    ## Mean Field methods
    def H_MF_nou(self, q_s, p_mf):
        return 1

    def H_MF_u(self, q_mf, p_mf, u_mf):
        # q_s, u_mf are vectors for ALL of the states, and controls
        # p_mf is a vector for ONLY the local states/costates
        # length of u_s must match number of terms here
        # some of the elements in u_s are quenched
        return self.H_MF_u_1(q_s, p_mf, u_s)*u_mf[0] + self.H_MF_u_2(q_s, p_mf, u_mf)*u_mf[1]

    def H_MF_u_1(self, q_mf, p_mf):
        return q_mf[0]*q_mf[1]
    
    def H_MF_u_2(self, q_mf, p_mf):
        return q_mf[1]
        
    def qp_rhs_H_mf(self, q_mf, p_mf, u_s):
        # remember that we want to propagate as much as possible together in the same rhs function for numerical purposes
        # remember that q_rhs here is w.r.t p_mf but p_rhs here is w.r.t q_s
        q_H_mf_dot = self.p_rhs_H_mf(q_mf, p_mf, u_mf, u_s)
        p_H_mf_dot = self.q_rhs_H_mf(q_mf, p_mf, u_mf, u_s)
        return np.concatenate([q_H_mf_dot, p_H_mf_dot])
    
    def q_rhs_H_mf(self, q_mf, p_mf, u_mf, u_s):
        # q_rhs_H_mf is the derivative wrt each of the local variables, so it will return something of dimension state_dim
        # q_rhs_H_mf_u returns the partial derivatives wrt each control, concatenated together
        q_rhs_H_mf_u = self.q_rhs_H_mf_u(q_mf, p_mf, u_mf)
        assert np.shape(q_rhs_H_mf_u)==(len(self.control_indices), self.state_dim) # first dimension should be number of controls, inner dimension should be state_dim
        q_rhs_H_mf_u_summed = sum([q_rhs_H_mf_u[i]*u_s[i] for i in range(len(u_s))])
        return self.q_rhs_H_mf_nou(q_mf, p_mf) + q_rhs_H_mf_u_summed
        
    def q_rhs_H_mf_u(self, q_mf, p_mf, u_mf):
        p_H_mf_u_dot_1 =  p_mf # or something
        return np.array([p_H_mf_u_dot_1])
    
    def p_rhs_H_mf(self, q_mf, p_mf, u_mf, u_s):
        # q_rhs_H_mf is the derivative wrt each of the local variables, so it will return something of dimension state_dim
        # q_rhs_H_mf_u returns the partial derivatives wrt each control, concatenated together
        p_rhs_H_mf_u = self.p_rhs_H_mf_u(q_mf, p_mf, u_mf)
        p_rhs_H_mf_u_summed = sum([p_rhs_H_mf_u[i]*u_s[i] for i in range(len(u_s))])
        return self.p_rhs_H_mf_nou(q_mf, p_mf) + p_rhs_H_mf_u_summed
        
    def p_rhs_H_mf_nou(self, q_mf, p_mf):
        return p_mf # or something

    def q_rhs_H_mf_nou(self, q_mf, p_mf):
        return p_mf

    def p_rhs_H_mf_u(self, q_mf, p_mf, u_mf):
        q_H_mf_u_dot = p_mf
        return np.array([q_H_mf_u_dot])

    def L_mf_q_dot(self, q_mf, q_mf_dot, u_mf):
        # q_mf_dot, q_mf (inputs) here will be vectors with ALL of the states
        # u_mf is a vector of ALL of the controls
        # extract q_s from q_mf
        
        # note that these methods must return vectors that are of local dimension - state_dim - even though they take in vectors of dimension for all the states
        # the user needs to be aware of the indices the correspond to each state
        
        def L_l_q_dot_agent_1(q_mf, q_mf_dot, u_mf):
            return np.array(q_mf[0])
        
        def L_l_q_dot_agent_2(q_mf, q_mf_dot, u_mf):
            return np.array(q_mf[1])
        
        L_mf_total_q_dot = np.zeros(self.state_dim)

        # agent 1
        L_mf_total_q_dot += L_l_q_dot_agent_1(q_mf, q_mf_dot, u_mf)

        # agent 2
        L_mf_total_q_dot += L_l_q_dot_agent_2(q_mf, q_mf_dot, u_mf)
        assert np.shape(L_mf_total_q_dot)[0] == self.state_dim, 'dimensions of L_mf_total_q_dot must match those of the local state, currently the dimensions are ' +str(np.shape(L_mf_total_q_dot)[0])
        return L_mf_total_q_dot

In [4]:
class Agent2:
    
    def __init__(self, blackboard, state_indices, control_indices):
        '''
        state_indices (list of integers): This list tells which states pertain to this agent. e.g. [1,2] would 
        tell us that states 1 and 2 pertain to this agent.
        
        A word oNn notation:  The notation used for the methods of the agent is:  
            - If it is a partial derivative: <denominator>_rhs_H_<type of hamiltonian (l, mf, or s)>_<nou or u>.  e.g., 
            "qp_rhs_H_l_u" denotes the partial derivative with respect to q and p of the terms in the local Hamiltonian that contain control variables.
            - If it is a hamiltonian: H_<type of hamiltonian (l, mf, or s)>_<nou or u>.  e.g. "H_mf_nou" denotes the mean field hamiltonian
            with terms not containing u.
        '''
        self.state_indices = state_indices
        self.control_indices = control_indices
        self.bb = blackboard

        # Inputs for numerical propagator
        # qp_vec is going to be [q_s, p_l, p_mf], so it will have dimension = 3*state_dim

        self.q_s_0 = np.array([0,2])
        self.p_l_0 = np.array([0,3])
        self.p_mf_0 = np.array([0,1])
        self.u_s_0 = np.array([0])
        self.qpu_vec = np.hstack([self.q_s_0, self.p_l_0, self.p_mf_0, self.u_s_0])
        self.state_dim = 2
        self.Gamma = 1 
        self.gamma = 1 # gets computed each time the agent is visited
        self.q_s_dot = np.array([0,1])  # must have same dimensions as q_s
        self.sync = None # gets Synchronizer class is initialized
        self.name='Agent2'
        
        # Inputs for numerical integration
        self.integrateTol = 10**-5
        self.integrateMaxIter = 400

        # Inputs for sliding window
        self.t_0 = 0
        self.T = 2
        self.K = 10

        self.t_terminal = 2
        self.n_s = 10

        self.validate_dimensions()

    def validate_dimensions(self):
        # TODO: move to parent class "SlidingWindow"
        assert len(self.state_indices) == self.state_dim, 'state dimensions are not consistent.  dimension of state indices is '+str(len(self.state_indices)) +' and state_dim is '+str(self.state_dim)
        assert len(self.control_indices) == len(self.u_s_0), 'control dimensions are not consistent.  dimension of control_indices is '+str(len(self.control_indices)) +' and len(u_0) is '+str(len(self.u_s_0))
        assert len(self.qpu_vec) == 3*self.state_dim + len(self.control_indices), ' control and state dimensions are not consistent with qpu_vec : length of qpu_vec is '+str(len(self.qpu_vec))+ ' and 3*self.state_dim + len(self.control_indices) is ' + str(3*self.state_dim + len(self.control_indices))
    
    '''
    TODO:  
    Add an assertion to check that the dimension of q_s_0, p_mf_0, and u_0:
        - the dimension of state_dim    
        - state_indices and control_indices set upon initiation of the Agent
        - 
    '''
    
    def L_l(self, q_s, q_s_dot, u_s):
        return 1
    
    def L_l_q_dot(self, q_s, q_s_dot, u_s):
        return q_s
    
    def H_l_nou(self, q_s, p_l, lambda_l):
        return 1

    def p_rhs_H_l_nou(self, q_s, p_l, lambda_l):
        return np.array(p_l)
    
    def q_rhs_H_l_nou(self, q_s, p_l, lambda_l):
        return np.array(p_l)

    def H_l_u(self, q_s, p_l):
        return 1
    
    def H_l(self, q_s, p_l, lambda_l, u_s):
        # used in "Construct local Hamiltonian of agent i"
        H_l = self.H_l_nou(q_s, p_l, lambda_l)
        H_l = H_l + self.H_l_u_1(q_s, p_s)*u_s[0]+ self.H_l_u_2(q_s, p_s)*u_s[1]
        return H_l
            
    def H_MF_u_1(self, q_mf, p_mf):
        return q_mf[0]*q_mf[1]
    
    def H_MF_u_2(self, q_mf, p_mf):
        return q_mf[1]
    
    def compute_lambdas(self, q_s, p_l, u_l):
        # not implemented yet
        return np.ones((1,self.state_dim))
    
    def p_rhs_H_l_u(self, q_s, p_l):
        return np.array(p_l)
    
    def q_rhs_H_l_u(self, q_s, p_l):
        return np.array(p_l)
    
    def H_l_D(self, q_lD, p_lD):
        return np.array(q_lD).dot(p_lD)
        
    def L_l_D(self, q_lD, p_lD):
        # return scalar
        return 1
        
    def L_l_D_q_Dot(self, q_lD, p_lD):
        # return 1 by state_dim, 1-D array
        # each q_lD is a 1-D array of size 1 by state_dim array
        return 1
    
    def qp_rhs(self, t, qp_vec, **kwargs):
        # u_s is constant (because of causality, remember?)
        u_s = kwargs['u_0']
        state_dim = kwargs['state_dim']
        q_mf = kwargs['q_mf']
        u_mf = kwargs['u_mf']
        
        # TODO:  get a kwargs working for lambda_l
        lambda_l = 0 # kwargs['lambda_l']
        q_s = qp_vec[:state_dim]
        p_l = qp_vec[state_dim:2*state_dim]
        p_mf = qp_vec[2*state_dim:]

        qp_rhs_H_mf = self.qp_rhs_H_mf(q_mf, p_mf, u_mf, u_s)
        p_rhs_H_mf = qp_rhs_H_mf[:state_dim]
        q_rhs_H_mf = qp_rhs_H_mf[state_dim:]
        
        qp_rhs_H_l = self.qp_rhs_H_l(q_s, p_l, u_s, lambda_l)
        q_rhs_H_l = qp_rhs_H_l[:state_dim]
        p_rhs_H_l = qp_rhs_H_l[state_dim:]

        q_s_dot = self.gamma*q_rhs_H_mf + (1-self.gamma)*q_rhs_H_l
        p_mf_dot = p_rhs_H_mf
        p_l_dot = -1*p_rhs_H_l
        
        return np.concatenate([q_s_dot, p_l_dot, p_mf_dot])
    
    def qp_rhs_H_l(self, q_s, p_l, u_s, lambda_l):
        #TODO: there is one lambda_l per constraint. need to work out dimensions.
        q_rhs_H_l = self.q_rhs_H_l_nou(q_s, p_l, lambda_l) + sum([self.q_rhs_H_l_u(q_s, p_l)*u_s_i for u_s_i in u_s])
        p_rhs_H_l = self.p_rhs_H_l_nou(q_s, p_l, lambda_l) + sum([self.p_rhs_H_l_u(q_s, p_l)*u_s_i for u_s_i in u_s])
        return np.concatenate([q_rhs_H_l, p_rhs_H_l])

    def u_rhs(self, t, u_vec, **kwargs):
        u_s = kwargs['u_0']
        state_dim = kwargs['state_dim']
        q_mf_dot = kwargs['q_mf_dot']
        q_s_dot = kwargs['q_s_dot']
        p_mf_dot = kwargs['p_mf_dot']
        q_mf = kwargs['q_mf']
        u_mf = kwargs['u_mf']
        p_mf = kwargs['p_mf']
        
        # don't have q_s_dot, q_mf_dot, or p_mf_dot
        def Beta_j(q_mf, p_mf, u_mf, u_s, q_s_dot, q_mf_dot, p_mf_dot, j):
            Beta_mf=[]
            Beta_l=[]
            for k in range(len(self.control_indices)):
                Beta_mf_k = self.H_mf_u[j]*(np.dot(self.q_rhs_H_mf_u(p_mf, q_mf, u_mf)[k], q_s_dot) + np.dot(self.p_rhs_H_mf_u(p_mf, q_mf, u_mf)[k], p_mf_dot)) +
                            self.H_mf_u[k]*(np.dot(self.q_rhs_H_mf_u(p_mf, q_mf, u_mf)[j], q_s_dot) + np.dot(self.p_rhs_H_mf_u(p_mf, q_mf, u_mf)[j], p_mf_dot))
                Beta_l_k = self.H_l_u[j]*(np.dot(self.q_rhs_H_l_u(q_s, p_l)[k], q_s_dot) + np.dot(self.p_rhs_H_l_u(q_s, p_l)[k], p_l_dot)) +
                            self.H_l_u[k]*(np.dot(self.q_rhs_H_l_u(q_s, p_l)[j], q_s_dot) + np.dot(self.p_rhs_H_l_u(q_s, p_l)[j], p_l_dot))
                
#         def alpha_j(q_mf, p_mf, u_mf, u_s, q_s_dot, q_mf_dot, p_mf_dot, j):
#             Beta_mf=[]
#             Beta_l=[]
#             for k in range(len(self.control_indices)):
#                 Beta_mf_k = self.H_mf_u[j]*(np.dot(self.q_rhs_H_mf_u(p_mf, q_mf, u_mf)[k], q_s_dot) + np.dot(self.p_rhs_H_mf_u(p_mf, q_mf, u_mf)[k], p_mf_dot)) +
#                             self.H_mf_u[k]*(np.dot(self.q_rhs_H_mf_u(p_mf, q_mf, u_mf)[j], q_s_dot) + np.dot(self.p_rhs_H_mf_u(p_mf, q_mf, u_mf)[j], p_mf_dot))
#                 Beta_l_k = self.H_l_u[j]*(np.dot(self.q_rhs_H_l_u(q_s, p_l)[k], q_s_dot) + np.dot(self.p_rhs_H_l_u(q_s, p_l)[k], p_l_dot)) +
#                             self.H_l_u[k]*(np.dot(self.q_rhs_H_l_u(q_s, p_l)[j], q_s_dot) + np.dot(self.p_rhs_H_l_u(q_s, p_l)[j], p_l_dot))
                
            
        def alpha_mf():
        
        def Beta_l()
        
        def alpha_l()
        
        qp_vec = kwargs['qp_vec']
        
        u_1_dot = -1*self.Gamma*(self.gamma*(alpha()))
        u_2_dot = 
        u_dot = np.concatenate([]) 

        return 


    ## Mean Field methods
    def H_MF_nou(self, q_mf, p_mf, u_mf):
        return 1

    def H_MF_u(self, q_mf, p_mf, u_mf):
        return 1
        
    def qp_rhs_H_mf(self, q_mf, p_mf, u_mf, u_s):
        # remember that we want to propagate as much as possible together in the same rhs function for numerical purposes
        # remember that q_rhs here is w.r.t p_mf but p_rhs here is w.r.t q_s
        q_H_mf_dot = self.p_rhs_H_mf(q_mf, p_mf, u_mf, u_s)
        p_H_mf_dot = self.q_rhs_H_mf(q_mf, p_mf, u_mf, u_s)
        return np.concatenate([q_H_mf_dot, p_H_mf_dot])
    
    def q_rhs_H_mf(self, q_mf, p_mf, u_mf, u_s):
        # q_rhs_H_mf is the derivative wrt each of the local variables, so it will return something of dimension state_dim
        # q_rhs_H_mf_u returns the partial derivatives wrt each control, concatenated together
        q_rhs_H_mf_u = self.q_rhs_H_mf_u(q_mf, p_mf, u_mf)
        assert np.shape(q_rhs_H_mf_u)==(len(self.control_indices), self.state_dim) # first dimension should be number of controls, inner dimension should be state_dim
        q_rhs_H_mf_u_summed = sum([q_rhs_H_mf_u[i]*u_s[i] for i in range(len(u_s))])
        return self.q_rhs_H_mf_nou(q_mf, p_mf) + q_rhs_H_mf_u_summed
        
    def q_rhs_H_mf_u(self, q_mf, p_mf, u_mf):
        # this method is will return a concatenation of all of the partial derivatives for each of the controls
        # each of the partial derivatives is of dimension state_dim
        # this means that this method will return a 2D array:
        #    - first dimension is the index of the control
        #    - second dimension is the index of the state
        p_H_mf_u_dot_1 = p_mf # must be of dimension state_dim
        q_rhs_H_mf_u = np.array([p_H_mf_u_dot_1])
        return q_rhs_H_mf_u

    def p_rhs_H_mf(self, q_mf, p_mf, u_mf, u_s):
        # q_rhs_H_mf is the derivative wrt each of the local variables, so it will return something of dimension state_dim
        # q_rhs_H_mf_u returns the partial derivatives wrt each control, concatenated together
        p_rhs_H_mf_u = self.p_rhs_H_mf_u(q_mf, p_mf, u_mf)
        p_rhs_H_mf_u_summed = sum([p_rhs_H_mf_u[i]*u_s[i] for i in range(len(u_s))])
        return self.p_rhs_H_mf_nou(q_mf, p_mf) + p_rhs_H_mf_u_summed
        
    def p_rhs_H_mf_nou(self, q_mf, p_mf):
        return  p_mf # or something

    def q_rhs_H_mf_nou(self, q_mf, p_mf):
        return  p_mf

    def p_rhs_H_mf_u(self, q_mf, p_mf, u_mf):
        q_H_mf_u_dot = p_mf
        return np.array([q_H_mf_u_dot])

    def L_mf_q_dot(self, q_mf, q_mf_dot, u_mf):
        # q_mf_dot, q_mf (inputs) here will be vectors with ALL of the states
        # u_mf is a vector of ALL of the controls
        # extract q_s from q_mf
        
        # note that these methods must return vectors that are of local dimension - state_dim - even though they take in vectors of dimension for all the states
        # the user needs to be aware of the indices the correspond to each state

        def L_l_q_dot_agent_1(q_mf, q_mf_dot, u_mf):
            return np.concatenate([np.array([q_mf[0]]),np.array([q_mf[1]])])
        
        def L_l_q_dot_agent_2(q_mf, q_mf_dot, u_mf):
            return np.concatenate([np.array([q_mf[0]]),np.array([q_mf[1]])])
        
        L_mf_total_q_dot = np.zeros(self.state_dim)

        # agent 1
        L_mf_total_q_dot += L_l_q_dot_agent_1(q_mf, q_mf_dot, u_mf)

        # agent 2
        L_mf_total_q_dot += L_l_q_dot_agent_2(q_mf, q_mf_dot, u_mf)

        assert np.shape(L_mf_total_q_dot)[0] == self.state_dim, 'dimensions of L_mf_total_q_dot must match those of the local state, currently the dimensions are ' +str(np.shape(L_mf_total_q_dot)[0])
        return L_mf_total_q_dot

### same for p

In [5]:
class Synchronizer:
    
    def __init__(self, agents, blackboard):
        self.agents = agents  # list of all agents. list with elements of class Agent
        self.bb = blackboard  # instance of class blackboard
        
        for agent in agents:
            self.bb.update_q_p_u_dict(agent)
            
        # add Synchronizer as to each agent
        for agent in agents:
            agent.sync = self
    
    def synchronize(self):
        # run synchronization by visiting each agent and running propagation
        for agent in self.agents:
            '''     
            1) run synchronized propagation - I think we only need one Agent instead of SlidingWindow now

            For each of the above 2 steps:
                - create sliding window instance
                - call "propagate_dynamics" on the sliding window instance

            get quenched values from blackboard
            '''
            # run propagation with keyword arguments state_dim_l, state_dim_mf
            q_ls_bars, p_ls_bars, p_mfs_bars, u_bars, windows = sliding_window(agent)

#     def L_mf_q_dot(self, q_mf, q_mf_dot, u_mf):
#         # q_mf_dot, q_mf (inputs) here will be vectors with ALL of the states
#         # u_mf is a vector of ALL of the controls
#         # extract q_s from q_mf
#         L_mf_total_q_dot = 0
#         for agent in self.agents:
#             # Evaluate L_q_dot for each of the agents
#             # Remember L_l_q_dot only takes in the local variables.  So, need to know the indices in q_mf which correspond to 
#             #...the states pertaining to this agent
#             q_s = np.array([q_mf[agent.state_indices[state_ix-1]-1] for state_ix in agent.state_indices])
#             q_s_dot = np.array([q_mf_dot[agent.state_indices[state_ix-1]-1] for state_ix in agent.state_indices])
#             u_s = np.array([u_mf[agent.control_indices[control_ix-1]-1] for control_ix in agent.control_indices])
    
#             L_l_q_dot = agent.L_l_q_dot(q_s, q_s_dot, u_s)
#             L_mf_total_q_dot = L_mf_total_q_dot + L_l_q_dot
            
#         return L_mf_total_q_dot

    def H_mf(self, q_mf, p_mf, u_mf, agent):
        # agent is an object of class agent.  It's the agent for which we are constructing H_mf.
        # start with H_mf = H_mf_nou, and then add the H_mf_u's

        H_mf = agent.H_mf_nou(q_mf, p_mf, u_mf)
        H_mf_u = agent.H_mf_u(q_mf, p_mf, u_mf)
        H_mf = H_mf + H_mf_u
        
        return H_mf
    

### Blackboard

In [6]:
class Blackboard:
    
    def __init__(self):
        '''
        values: list of integers state_indices state_indices
        q_p_u_dict is a dictionary which maps 'q', 'p', 'u', to a dictionary of index-value pairs: local values for q, p, and u for this agent.  
                        blackboard holds all of the most recent local values, e.g.
                        {'q_s': {'1':3, '2':0}, 'p_mf': {'1':3, '2': 2}, 'u': {'1': 0}}
                        It doesn't care which agent updated them most recently.  It only needs to know which values to update.
        q_p_u_dict initially will be filled.
        '''
        ## TODO:  _s should really be called _mf because it contains all of the states/controls.
        self.q_p_u_dict = {'q_s':{}, 'p_l':{}, 'p_mf':{}, 'u_s':{}, 'q_s_dot':{}}
        self.agents=[]
        
    def update_q_p_u_dict(self, agent):
        '''This method should be called after propagation of each agent
        Inputs:
            agent (instance of class Agent): this is the agent whose values we are updating
        Outputs:
            No outputs.  This method just updates the attributes of the blackboard,
            just update the dictionary, agent_q_p_u_dict.
        '''
        # Determine which states pertain to this agent and replace the old values with new
        for state_ix in agent.state_indices:
            self.q_p_u_dict['q_s'][str(state_ix)] = agent.qpu_vec[:agent.state_dim][state_ix-1]
            self.q_p_u_dict['p_l'][str(state_ix)] = agent.qpu_vec[agent.state_dim:2*agent.state_dim][state_ix-1]
            self.q_p_u_dict['p_mf'][str(state_ix)] = agent.qpu_vec[2*agent.state_dim:3*agent.state_dim][state_ix-1]
            self.q_p_u_dict['q_s_dot'][str(state_ix)] = agent.q_s_dot[state_ix-1]
            
        for control_ix in agent.control_indices:
            self.q_p_u_dict['u_s'][str(control_ix)] = agent.qpu_vec[3*agent.state_dim:][control_ix-1]
        
        # update sensor values "q_l, q_l_dot, u_s"
#         self.q_p_u_dict['q_1'] = [] unnecessary because identical to q_s
#         self.q_p_u_dict['q_1_dot'] = []  # numerically estimated, or available from sensors
#         self.q_p_u_dict['u_s'] = []  # unnecessary because same as "u_s" above
        
        # add agent if not already added 
        if agent not in self.agents:
            self.agents.append(agent)


## Small test for two agents

- Add agents to blackboard and meanfield
- Run synchronizer to visit the agents

In [7]:
bb = Blackboard()
# mean_field = MeanField()

myAgent=Agent(bb, state_indices=[1], control_indices=[1])
myAgent2=Agent2(bb, state_indices=[1,2], control_indices=[1])

In [8]:
bb.update_q_p_u_dict(myAgent)
bb.update_q_p_u_dict(myAgent2)

In [9]:
q_p_u_dict = bb.q_p_u_dict

In [10]:
q_p_u_dict
# this only tells us the values of each state/costate.  Does not tell us which ones correspond to which agent
# state_indices attribute of each agent tells us which states pertain to the agent

{'p_l': {'1': 0, '2': 3},
 'p_mf': {'1': 0, '2': 1},
 'q_s': {'1': 0, '2': 2},
 'q_s_dot': {'1': 0, '2': 1},
 'u_s': {'1': 0}}

In [11]:
agents = [myAgent, myAgent2]
sync = Synchronizer(agents, bb)

In [12]:
q_mf, q_mf_dot, u_mf = construct_mf_vectors(myAgent2)
q_s, q_s_dot, u_s = construct_local_vectors(myAgent2)

# this should return a 2-D array
# then, for each of the arrays, we multiply by a scalar value of the control, and add them together.
# So,  myAgent2.q_rhs_H_mf(q_mf, q_s, u_mf) should return something of dimension equal to state_dim
print myAgent2.q_rhs_H_mf_u(q_mf, q_s, u_mf)

print myAgent2.qp_rhs_H_mf(q_mf, q_s, u_mf, u_s) # this should return something of dimension state_dim*2
print myAgent2.q_rhs_H_mf(q_mf, q_s, u_mf, u_s) # this should return something of dimension state_dim

[[0. 2.]]
[0. 2. 0. 2.]
[0. 2.]


In [13]:
# this should return a 2-D array
# then, for each of the arrays, we multiply by a scalar value of the control, and add them together.
# So,  myAgent2.q_rhs_H_mf(q_mf, q_s, u_mf) should return something of dimension equal to state_dim
print myAgent2.p_rhs_H_mf_u(q_mf, q_s, u_mf)

print myAgent2.qp_rhs_H_mf(q_mf, q_s, u_mf, u_s) # this should return something of dimension state_dim*2
print myAgent2.p_rhs_H_mf(q_mf, q_s, u_mf, u_s) # this should return something of dimension state_dim

[[0. 2.]]
[0. 2. 0. 2.]
[0. 2.]


### Small test for update_q_mf

In [14]:

q_mf=np.array([1000,1000])
q_mf = update_q_mf(q_mf, q_s, myAgent2)
assert q_mf[0] == np.array([0]), 'something went wrong'
assert q_mf[1] == np.array([2]), 'something went wrong'

### Try a very small test

Steps to test qp_rhs and u_rhs:

- Give some arbitrary initial conditions
- create Agent
- create Blackboard
- create MeanField
- connect those three things above.
- somehow create a sliding window instance, or at least wrangle the Agent into a Sliding Window instance

In [15]:
# forgot to pass in 'u_s' as a keyword argument
qpu_vec, q_ss_bar, p_ls_bar, p_mfs_bar, u_bar, q_ss, p_ls, p_mfs, us, window = propagate_dynamics(myAgent)
# forgot to pass in 'u_s' as a keyword argument
# qpu_vec2, q_ss_bar2, p_ls_bar2, p_mfs_bar2, u_bar2, q_ss2, p_ls2, p_mfs2, us2 = propagate_dynamics(myAgent2)

In [16]:
print qpu_vec, q_ss_bar, p_ls_bar, p_mfs_bar, u_bar, q_ss, p_ls, p_mfs, us

# print qpu_vec2, q_ss_bar2, p_ls_bar2, p_mfs_bar2, u_bar2, q_ss2, p_ls2, p_mfs2, us2

[0. 0. 0. 0.] [0.] [0.] [0.] [0.] [array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.])] [array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.])] [array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.])] [array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), array([0.]), 

### Try another very small test

- run propagation for Agent using multiple windows so we can test using initial values, etc.
- and we can also test all of the methods for the agent rather than only the one for 


### Get sliding window working for multiple windows

In [17]:
# write helper function to take in sliding window, and q_p_u_dict, and return the q_s

In [18]:
q_p_u_dict['q_s'].items()

[('1', 0), ('2', 2)]

In [19]:
def sliding_window(sliding_window_instance):
    ''' 
    This method runs the propagation for a single agent.  Corresponding to the flow chart it runs:
        - Read from blackboard to get the following observation measured at time t_0, and onwards
        - construct quenched mean field for Hamiltonian agent i
        - Setup initial conditions for L_MF and p_MF
        - Construct agent synchronized Hamiltonian and partial derivatives
    
    Inputs:
    The only input is sliding_window_instance, but we use the following attributes of the sliding_window_instance:
        t_0 (int): Initial time to start propagating dynamics
        T (int): End time of propagating dynamics
        q_0 (np.array): initial values of state vector
        p_0 (np.array): initial values of costate vector
        u_0 (np.array): initial values of control vector
        state_dim (int): number of states
        Gamma (float): algorithmic parameter for Riemann descent algorithm
        t_terminal (int): time marking termination of control law propagator algorithm
    Outputs:
        q_bars, p_bars, u_bars (list of np.arrays): implemented state/costate/control values for entire propagator.
    '''
    
    t_0, T, K, state_dim,t_terminal = sliding_window_instance.t_0, sliding_window_instance.T, sliding_window_instance.K,  sliding_window_instance.state_dim, sliding_window_instance.t_terminal
    q_ls_bars, p_ls_bars, p_mfs_bars, u_bars, windows = [], [], [], [], []
    t = t_0 # wall clock time
    
    # Read from blackboard to get the following observations measured at time t_0
    q_s_0, q_s_dot_0, u_s_0 = construct_local_vectors(sliding_window_instance)
    q_mf, q_mf_dot, u_mf = construct_mf_vectors(sliding_window_instance)

    # now pick out the individual states that we need to make q_s and q_s_dot
    qpu_vec = sliding_window_instance.qpu_vec
    state_dim = sliding_window_instance.state_dim
    # a note on q_s_dot - normally I understand that this would come from the sensors, ...
    # ...but for now get it from q_mf_dot from the blackboard, and just select if from the states that pertain to this agent
    # construct quenched mean field for Hamiltonian agent i
    # this happens inside of the class Synchronize method
    
    # If control is physical, then we should use the physical value here for initial condition
    # IF not, then we can use the average u, averaged over the previous window.
    # For now, use the value from the blackboard
    q_l_D_dot_0 = q_s_dot_0
    q_l_D_0 = q_s_0

    # set initial conditions using values from blackboard retrieved above
    # set initial conditions for local Hamiltonian of agent i
    p_l_0 = sliding_window_instance.L_l_q_dot(q_s_0, q_s_dot_0, u_s_0) # compute using Dirac compatibility
    p_l_D_0 = sliding_window_instance.L_l_D_q_Dot(q_l_D_0, q_l_D_dot_0) # compute using Dirac compatibility
    H_l_D_0 = sliding_window_instance.H_l_D(q_l_D_0, p_l_D_0)
    
    # setup initial condition for p_mf
    p_mf_0 = sliding_window_instance.L_mf_q_dot(q_mf, q_mf_dot, u_mf)
    
    # now construct qpu_vec 
    sliding_window_instance.qpu_vec = np.concatenate([q_s_0, p_l_0, p_mf_0, u_s_0]) # fill in with blackboard values for q and u, but for p, must be computed
    # Construct local Hamiltonian of agent i
    lambdas = sliding_window_instance.compute_lambdas(q_s_0, p_l_0, u_s_0)

    while t < sliding_window_instance.t_terminal:
        
        # for the times, propagate_dynamics needs: t_0, T, and K.  T and K can come from the sliding_window_instance
        #...t_0 will be passed in.  t_0 is the start of the window.

        # this propagates a single window
        # inside of propagate dynamics
        qpu_vec, q_ls_bar, p_ls_bar, p_mfs_bar, u_bar, q_ls, p_ls, p_mfs, us, window = propagate_dynamics(sliding_window_instance)
        # qs, ps, and us will go to Mean Field somehow
        
        q_ls_bars.append(q_ls_bar)
        p_ls_bars.append(p_ls_bar)
        p_mfs_bars.append(p_mfs_bar)
        u_bars.append(u_bar)
        windows.append(window)

        t+=1
    # update blackboard
    bb.update_q_p_u_dict(sliding_window_instance)
    
    return q_ls_bars, p_ls_bars, p_mfs_bars, u_bars, windows


In [20]:
myAgent.qpu_vec

array([0, 0, 0, 0])

In [21]:
q_ls_bars, p_ls_bars, p_mfs_bars, u_bars, windows = sliding_window(myAgent)

In [26]:
myAgent2.q_rhs_H_mf_u(p_mfs_bar[0],q_mf,u_mf)[k]

array([[0, 2]])

In [22]:
qpu_vec2, q_ss_bar2, p_ls_bar2, p_mfs_bar2, u_bar2, q_ss2, p_ls2, p_mfs2, us2 = propagate_dynamics(myAgent2)

AttributeError: Agent2 instance has no attribute 'u_rhs'

In [ ]:
sync.synchronize()